In [7]:
import os
import random
import numpy as np
import pandas as pd
import seaborn as sns
import urllib.request
import zipfile
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm import tqdm
from random import shuffle
from IPython.display import SVG
from tensorflow.keras.utils import model_to_dot
from tensorflow.keras.utils import plot_model
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D

%matplotlib inline

In [8]:
TEST_SIZE = 0.5
RANDOM_STATE = 2018
BATCH_SIZE = 64
NO_EPOCHS = 10
NUM_CLASSES = 2
SAMPLE_SIZE = 20000
IMG_SIZE = 224

In [9]:
PATH = "/content/drive/MyDrive/Colab Notebooks/"  # Or your desired path
train_image_path = os.path.join(PATH, "train.zip")
test_image_path = os.path.join(PATH, "test.zip")

In [10]:
def download_progress(block_num, block_size, total_size):
    downloaded = block_num * block_size
    percent = min(100, downloaded * 100 / total_size)
    print(f"\rBaixando: {percent:.2f}%", end='')


In [11]:
# URL do dataset
url = "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip"
local_zip = "cats-and-dogs.zip"

# Download do arquivo com progresso
print("Iniciando download...")
urllib.request.urlretrieve(url, local_zip, reporthook=download_progress)
print("\nDownload concluído!")

# Criar diretório para extração
os.makedirs('IMAGENS', exist_ok=True)

# Extração do arquivo ZIP
print("Extraindo arquivos...")
with zipfile.ZipFile(local_zip, 'r') as zip_ref:
    zip_ref.extractall('IMAGENS')
print("Extração concluída!")

Iniciando download...
Baixando: 100.00%
Download concluído!
Extraindo arquivos...


KeyboardInterrupt: 

In [ ]:
root = 'IMAGENS/PetImages'
exclude = ['Thumbs.db']
corrupted_files = [
    os.path.join(root, "Cat/666.jpg"),
    os.path.join(root, "Dog/11702.jpg")
]

# Remover arquivos corrompidos
for file in corrupted_files:
    if os.path.exists(file):
        os.remove(file)

In [ ]:
categories = [x[0] for x in os.walk(root) if x[0]][1:]
categories = [c for c in categories if c not in [os.path.join(root, e) for e in exclude]]

print(categories)

In [ ]:
def label_pet_image_one_hot_encoder(img):
    pet = img.split('.')[-3]
    if pet == 'cat': return [1,0]
    elif pet == 'dog': return [0,1]

In [ ]:
def process_data(data_image_list, DATA_FOLDER, isTrain=True):
    data_df = [] # Initialize data_df as an empty list
    for img in tqdm(data_image_list):
        path = os.path.join(DATA_FOLDER,img)
        if(isTrain):
            label = label_pet_image_one_hot_encoder(img)
        else:
            label = img.split('.')[0]
        img = cv2.imread(path,cv2.IMREAD_COLOR)

        # Check if the image was loaded successfully
        if img is None:
            print(f"Warning: Could not read image at path: {path}")
            continue  # Skip this image if it's not readable

        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        data_df.append([np.array(img),np.array(label)])
    shuffle(data_df)
    return data_df

In [ ]:
train_image_list = []
for folder in ['Cat', 'Dog']:
    folder_path = os.path.join(root, folder)
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):  # Add any other image extensions if needed
            train_image_list.append(os.path.join(folder, filename))  # Store relative paths

In [ ]:
def plot_image_list_count(data_image_list):
    labels = []
    for img in data_image_list:
        # Check if the image name has enough parts after splitting
        parts = img.split('.')
        if len(parts) >= 3:  # Make sure there are at least 3 parts
            labels.append(parts[-3])  # Now it's safe to access [-3]
    sns.countplot(labels)
    plt.title('Cats and Dogs')

In [ ]:
def label_pet_image_one_hot_encoder(img):
    # Split the path using os.path.sep to handle different path separators
    parts = img.split(os.path.sep)
    # Get the second to last part, which should be 'Cat' or 'Dog'
    pet = parts[-2]
    if pet == 'Cat':
        return [1, 0]
    elif pet == 'Dog':
        return [0, 1]

In [ ]:
def show_images(data, isTest=False):
    f, ax = plt.subplots(5,5, figsize=(15,15))
    for i,data in enumerate(data[:25]):
        img_num = data[1]
        img_data = data[0]
        label = np.argmax(img_num)
        if label  == 1:
            str_label='Dog'
        elif label == 0:
            str_label='Cat'
        if(isTest):
            str_label="None"
        ax[i//5, i%5].imshow(img_data)
        ax[i//5, i%5].axis('off')
        ax[i//5, i%5].set_title("Label: {}".format(str_label))
    plt.show()
train = process_data(train_image_list, root)
show_images(train)

In [ ]:
# Create a list to store the paths of the test images
test_image_list = []

# Assuming all images are in the root directory and you want to split them for testing
# You can modify this logic based on your actual data structure
for folder in ['Cat', 'Dog']:
    folder_path = os.path.join(root, folder)
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            # Example: Use a percentage of images for testing (e.g., 20%)
            if random.random() < 0.20:  # Adjust the percentage as needed
                test_image_list.append(os.path.join(folder, filename))
            else:
                train_image_list.append(os.path.join(folder, filename))

# Now you can call process_data with test_image_list
# You may need to adjust isTrain=True if you want to process labels for the test set
test = process_data(test_image_list, root, isTrain=True)

In [ ]:
show_images(test,True)

In [ ]:
X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
y = np.array([i[1] for i in train])

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            print(f"Memory growth enabled for GPU: {gpu}")
    except RuntimeError as e:
        print(e)

In [ ]:
model = Sequential()
model.add(ResNet50(include_top=False, pooling='max', weights='imagenet'))  # Use 'imagenet' for pre-trained weights
model.add(Dense(NUM_CLASSES, activation='softmax'))
model.layers[0].trainable = True

In [ ]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)

In [ ]:
train_model = model.fit(X_train, y_train,
                  batch_size=BATCH_SIZE,
                  epochs=NO_EPOCHS,
                  verbose=1,
                  validation_data=(X_val, y_val))

In [ ]:
def plot_accuracy_and_loss(train_model):
    hist = train_model.history
    acc = hist['acc']
    val_acc = hist['val_acc']
    loss = hist['loss']
    val_loss = hist['val_loss']
    epochs = range(len(acc))
    f, ax = plt.subplots(1,2, figsize=(14,6))
    ax[0].plot(epochs, acc, 'g', label='Training accuracy')
    ax[0].plot(epochs, val_acc, 'r', label='Validation accuracy')
    ax[0].set_title('Training and validation accuracy')
    ax[0].legend()
    ax[1].plot(epochs, loss, 'g', label='Training loss')
    ax[1].plot(epochs, val_loss, 'r', label='Validation loss')
    ax[1].set_title('Training and validation loss')
    ax[1].legend()
    plt.show()
plot_accuracy_and_loss(train_model)

In [ ]:
score = model.evaluate(X_val, y_val, verbose=0)
print('Validation loss:', score[0])
print('Validation accuracy:', score[1])

In [ ]:
#get the predictions for the test data
predicted_classes = model.predict_classes(X_val)
#get the indices to be plotted
y_true = np.argmax(y_val,axis=1)

In [ ]:
correct = np.nonzero(predicted_classes==y_true)[0]
incorrect = np.nonzero(predicted_classes!=y_true)[0]

In [ ]:
target_names = ["Class {}:".format(i) for i in range(NUM_CLASSES)]
print(classification_report(y_true, predicted_classes, target_names=target_names))

In [ ]:
f, ax = plt.subplots(5,5, figsize=(15,15))
for i,data in enumerate(test[:25]):
    img_num = data[1]
    img_data = data[0]
    orig = img_data
    data = img_data.reshape(-1,IMG_SIZE,IMG_SIZE,3)
    model_out = model.predict([data])[0]

    if np.argmax(model_out) == 1:
        str_predicted='Dog'
    else:
        str_predicted='Cat'
    ax[i//5, i%5].imshow(orig)
    ax[i//5, i%5].axis('off')
    ax[i//5, i%5].set_title("Predicted:{}".format(str_predicted))
plt.show()

In [ ]:
pred_list = []
img_list = []
for img in tqdm(test):
    img_data = img[0]
    img_idx = img[1]
    data = img_data.reshape(-1,IMG_SIZE,IMG_SIZE,3)
    predicted = model.predict([data])[0]
    img_list.append(img_idx)
    pred_list.append(predicted[1])